In [548]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error

import random

%matplotlib inline

In [549]:
path = '/Users/mjschillawski/Google Drive/Data/generalassembly/data/'
filename = 'iowa_liquor_sales_proj_2.csv'

## Load the data into a DataFrame
data = pd.read_csv(path+filename)

## Transform the dates if needed, e.g.
data["Date"] = pd.to_datetime(data["Date"])

/Users/mjschillawski/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [550]:
def DataInspect(dataframe):
    '''Original function (previously called eda) created by Ritika Bhasker
       Good first step when starting any project. Provides overview of
       dataset including missing values, duplicates and types.
       Takes a Pandas dataframe as the argument.'''
    print("Dataframe Shape:", dataframe.shape,"\n")
    print("Duplicate Rows:", dataframe.duplicated().sum(),"\n") #Added this
    print("Dataframe Types \n\n", dataframe.dtypes,"\n")    
    print("Missing Values \n\n", dataframe.isnull().sum(),"\n")
    print("Dataframe Describe \n\n", dataframe.describe(include='all'),"\n")


    print('Unique Values by Variable')
    for item in dataframe:
        print(item,':',dataframe[item].nunique())

In [551]:
DataInspect(data)

Dataframe Shape: (2709552, 24) 

Duplicate Rows: 0 

Dataframe Types 

 Invoice/Item Number              object
Date                     datetime64[ns]
Store Number                      int64
Store Name                       object
Address                          object
City                             object
Zip Code                         object
Store Location                   object
County Number                   float64
County                           object
Category                        float64
Category Name                    object
Vendor Number                     int64
Vendor Name                      object
Item Number                       int64
Item Description                 object
Pack                              int64
Bottle Volume (ml)                int64
State Bottle Cost                object
State Bottle Retail              object
Bottles Sold                      int64
Sale (Dollars)                   object
Volume Sold (Liters)            float64
Volume S

Invoice/Item Number : 2709552
Date : 284
Store Number : 1403
Store Name : 1397
Address : 1398
City : 386
Zip Code : 743
Store Location : 1745
County Number : 99
County : 99
Category : 89
Category Name : 73
Vendor Number : 159
Vendor Name : 160
Item Number : 3865
Item Description : 3029
Pack : 19
Bottle Volume (ml) : 34
State Bottle Cost : 1471
State Bottle Retail : 1525
Bottles Sold : 275
Sale (Dollars) : 11456
Volume Sold (Liters) : 544
Volume Sold (Gallons) : 538


In [552]:
data.rename(columns={'Date':'date', 'Store Number':'storenum', 'City':'city', 
                     'Zip Code':'zipcode', 'County Number':'countynum', 
                     'County':'county',
                     'Category':'category', 'Category Name':'categoryname', 
                     'Vendor Number':'vendornum',
                     'Item Number':'itemnum','Item Description':'itemdescription', 
                     'Bottle Volume (ml)':'bottlevol_ml', 
                     'State Bottle Cost':'statebottlecost',
                     'State Bottle Retail':'statebottleretail', 
                     'Bottles Sold':'bottlessold', 
                     'Sale (Dollars)':'sale_dollars',
                     'Volume Sold (Liters)':'volsold_liters', 
                     'Volume Sold (Gallons)':'volsold_gallons'},inplace=True)

In [553]:
#Drop duplicate rows

def dropduplicates(df):
    print('Original shape:',df.shape)
    temp =  df.drop(df[df.duplicated()].index,axis=0)
    print('Dropped duplicates, new shape: ',temp.shape)
    return temp

data = dropduplicates(data)

Original shape: (2709552, 24)
Dropped duplicates, new shape:  (2709552, 24)


In [554]:
#Remove dollar signs and convert to numeric

bad_col = ['zipcode','statebottlecost','statebottleretail','sale_dollars']

for item in bad_col:
    data[item] = data[item].str.replace('$','')
    data[item] = data[item].apply(pd.to_numeric)
    
data.head()

,Invoice/Item Number,date,storenum,Store Name,Address,city,zipcode,Store Location,countynum,county,...,itemnum,itemdescription,Pack,bottlevol_ml,statebottlecost,statebottleretail,bottlessold,sale_dollars,volsold_liters,volsold_gallons
0,S29198800001,2015-11-20,2191,Keokuk Spirits,1013 MAIN,KEOKUK,52632,"1013 MAIN\nKEOKUK 52632\n(40.39978, -91.387531)",56.0,Lee,...,297,Templeton Rye w/Flask,6,750,18.09,27.14,6,162.84,4.50,1.19
1,S29195400002,2015-11-21,2205,Ding's Honk And Holler,900 E WASHINGTON,CLARINDA,51632,"900 E WASHINGTON\nCLARINDA 51632\n(40.739238, ...",73.0,Page,...,297,Templeton Rye w/Flask,6,750,18.09,27.14,12,325.68,9.00,2.38
2,S29050300001,2015-11-16,3549,Quicker Liquor Store,1414 48TH ST,FORT MADISON,52627,"1414 48TH ST\nFORT MADISON 52627\n(40.624226, ...",56.0,Lee,...,249,Disaronno Amaretto Cavalli Mignon 3-50ml Pack,20,150,6.40,9.60,2,19.20,0.30,0.08
3,S28867700001,2015-11-04,2513,Hy-Vee Food Store #2 / Iowa City,812 S 1ST AVE,IOWA CITY,52240,812 S 1ST AVE\nIOWA CITY 52240\n,52.0,Johnson,...,237,Knob Creek w/ Crystal Decanter,3,1750,35.55,53.34,3,160.02,5.25,1.39
4,S29050800001,2015-11-17,3942,Twin Town Liquor,104 HIGHWAY 30 WEST,TOLEDO,52342,"104 HIGHWAY 30 WEST\nTOLEDO 52342\n(41.985887,...",86.0,Tama,...,249,Disaronno Amaretto Cavalli Mignon 3-50ml Pack,20,150,6.40,9.60,2,19.20,0.30,0.08


In [555]:
DataInspect(data)

Dataframe Shape: (2709552, 24) 

Duplicate Rows: 0 

Dataframe Types 

 Invoice/Item Number            object
date                   datetime64[ns]
storenum                        int64
Store Name                     object
Address                        object
city                           object
zipcode                        object
Store Location                 object
countynum                     float64
county                         object
category                      float64
categoryname                   object
vendornum                       int64
Vendor Name                    object
itemnum                         int64
itemdescription                object
Pack                            int64
bottlevol_ml                    int64
statebottlecost               float64
statebottleretail             float64
bottlessold                     int64
sale_dollars                  float64
volsold_liters                float64
volsold_gallons               float64
dtype: object 



In [556]:
data.drop(['Invoice/Item Number','Store Name','Address','Store Location','vendornum','Vendor Name','Pack'],axis=1,inplace=True)
data.shape

(2709552, 17)

In [557]:
#Extract unique City/Zipcode/CountyNum/County tuples from list
#Drop if CountyNum and County are null
#https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/

##change this
counties = data[data[['city','zipcode','countynum',
                      'county']].duplicated()==False][['city','zipcode',
                                                       'countynum','county']]
counties = counties[counties['countynum'].notnull() & counties['county'].notnull()]
counties['county'].nunique()

99

In [558]:
#Check for duplicated City/Zipcode tuples assigned to multiple counties.
counties[counties.duplicated(['city','zipcode'],False)].sort_values('city')

,city,zipcode,countynum,county
290,ACKLEY,50601,42.0,Hardin
23260,ACKLEY,50601,94.0,Webster
2392677,BETTENDORF,52722,82.0,Scott
2397595,BETTENDORF,52722,48.0,Iowa
254,BETTENDORF,52722,82.0,Scott
678,BETTENDORF,52722,48.0,Iowa
2394018,CLIVE,50325,25.0,Dallas
2397310,CLIVE,50325,77.0,Polk
1682,CLIVE,50325,25.0,Dallas
420,CLIVE,50325,77.0,Polk


# Impute missing data

In [559]:
full_missing = data[np.logical_and(data['countynum'].isnull(), data['county'].isnull())]
print(len(full_missing))
missing_deduped = full_missing[full_missing[['storenum','city','zipcode',
                        'countynum','county']].duplicated()==False][['storenum',
                                                                     'city','zipcode','countynum','county']]
missing_deduped.head()
print(len(missing_deduped))

10913
65


In [618]:
#Identify list of unique stores missing County, Countynum
#THEN:
##Check for other records from store with county, countynum not missing --> replace for all store records
#THEN:
##Check for other city/zip matches
###IF only 1 county returned --> replace for all store records
###ELSE randomly select county --> replace for all store records

random.seed(19871006)

#First, find the set of data missing county and countynum
full_missing = data[np.logical_and(data['countynum'].isnull(), data['county'].isnull())]

start = len(full_missing)
print('Missing values:',start)

#Second, dedupe the list to find the unique stores missing those data
missing_deduped = full_missing[full_missing[['storenum','city','zipcode',
                        'countynum','county']].duplicated()==False][['storenum',
                                                                     'city','zipcode','countynum','county']]

#initialize counters
store_match = 0
exact_county_match = 0
hotdeck = 0
no_match = 0

#iterate over deduped list
for i in range(len(missing_deduped)):
    
    new_county = ''
    new_countynum = ''
    
    #scan full dataset for other store records
    temp_county = data[data['storenum']==missing_deduped.iat[i,0]]['county'].value_counts()
    
    #if value_counts has non-null values, will return shape >= 1
    #if shape equals 1, we can grab the store's correct county, countynum and replace it
    if temp_county.shape[0] == 1:
        new_county = temp_county.index[0]
        
        temp_countynum = data[data['storenum']==missing_deduped.iat[i,0]]['countynum'].value_counts()
        new_countynum = temp_countynum.index[0]
        

        #get the index of the records with missing county, countynum that match the storenum
        #update those records with the correct data
        for n in full_missing[full_missing['storenum']==missing_deduped.iat[i,0]].index:
            data.iat[n,4] = new_countynum
            data.iat[n,5] = new_county
            
            store_match += 1
    
    
    else:
        #pull out important keys
        index = i
        store_missing = missing_deduped.iat[i,0]
        city_missing = missing_deduped.iat[i,1]
        zipcode_missing = missing_deduped.iat[i,2]
        
        print('string:',store_missing,city_missing,zipcode_missing)
        
        #define set of possible replacement values
        replacement = counties[(counties['city']==city_missing) & 
                                (counties['zipcode']==zipcode_missing)]
        print(replacement.head())

        #if only 1 possible replacement, set new values for all records with storenum
        if len(replacement) == 1:
            new_countynum = replacement.iat[0,1]
            new_county = replacement.iat[0,2]

            for n in full_missing[full_missing['storenum']==missing_deduped.iat[i,0]].index:
                data.iat[n,4] = new_countynum
                data.iat[n,5] = new_county

                exact_county_match += 1

        #if more than 1 possible replacement, randomly select 1 value, apply to all records with storenum
        elif len(replacement) > 1:
            srs = replacement.sample(1,axis=0)
            new_countynum = srs.iat[0,1]
            new_county = srs.iat[0,2]

            for n in full_missing[full_missing['storenum']==missing_deduped.iat[i,0]].index:
                data.iat[n,4] = new_countynum
                data.iat[n,5] = new_county

                hotdeck += 1
        
        else:
            print('')
            print('No matches found. Cannot impute.')
            print(store_missing,city_missing,zipcode_missing)
            no_match += 1

#change this
end = len(data[np.logical_and(data['countynum'].isnull(), data['county'].isnull())])

print('')
print('*** Imputation Report ***')
print('')
print('Remaining null values:',end)
print('Filled by from other records from same store:',store_match)
print('Filled by exact county match by city/zip:',exact_county_match)
print('Filled by hotdecking:',hotdeck)
print('No matches found:',no_match)
print('')
print('***end***')

Missing values: 178

No matches found. Cannot impute.
5217 SEYMOUR 52590

No matches found. Cannot impute.
5217 SEYMOUR 52590

No matches found. Cannot impute.
5223 TABOR 51653

No matches found. Cannot impute.
5216 RUNNELLS 50237

*** Imputation Report ***

Remaining null values: 178
Filled by from other records from same store: 0
Filled by exact county match by city/zip: 0
Filled by hotdecking: 0
No matches found: 4

***end***


In [44]:
#subset 2016 data
#for predictions later
test2016 = data[data['date'] > '2015-12-31']
print(test2016.shape)

#isolate 2016 data from 2015 training data
train2015 = data[data['date'] < '2016-01-01']
print(train2015.shape)

print(test2016.shape[0] + train2015.shape[0])

(525069, 17)
(2184483, 17)
2709552


In [46]:
print(train2015['date'].describe())
print('')
print(test2016['date'].describe())

count                 2184483
unique                    227
top       2015-12-01 00:00:00
freq                    15588
first     2015-01-05 00:00:00
last      2015-12-31 00:00:00
Name: date, dtype: object

count                  525069
unique                     57
top       2016-01-04 00:00:00
freq                    13251
first     2016-01-04 00:00:00
last      2016-03-31 00:00:00
Name: date, dtype: object


# Begin 2015 EDA / Transformations

In [285]:
#Annual 2015 totals by store
stores2015 = train2015.groupby('storenum')[['bottlevol_ml','statebottlecost',
                                            'statebottleretail','bottlessold',
                                            'sale_dollars','volsold_liters',
                                            'volsold_gallons']].sum()
stores2015.head()

,bottlevol_ml,statebottlecost,statebottleretail,bottlessold,sale_dollars,volsold_liters,volsold_gallons
storenum,,,,,,,
2106,4643675,55183.96,82829.51,99998,1434369.85,93986.47,24832.44
2113,1313875,14469.12,21725.74,6483,85763.42,6500.83,1717.17
2130,3647275,40580.91,60908.96,72562,1108184.99,65562.88,17321.29
2152,1513250,12311.88,18507.48,5928,72080.36,6164.92,1628.42
2178,2739775,26665.46,40070.07,20504,277987.96,21719.49,5737.98


In [286]:
##define target vector
y2015 = stores2015[['sale_dollars']]
y2015.head()

,sale_dollars
storenum,
2106,1434369.85
2113,85763.42
2130,1108184.99
2152,72080.36
2178,277987.96


In [287]:
#2015 County Report
county2015 = train2015.groupby('county')[['bottlevol_ml','statebottlecost',
                                          'statebottleretail','bottlessold',
                                          'sale_dollars','volsold_liters',
                                          'volsold_gallons']].sum()
county2015.head()

,bottlevol_ml,statebottlecost,statebottleretail,bottlessold,sale_dollars,volsold_liters,volsold_gallons
county,,,,,,,
Adair,4476500,40309.73,60565.89,35049,424750.82,33957.85,8969.34
Adams,1757403,18103.82,27171.39,8446,100596.80,7547.62,1992.02
Allamakee,9149175,85870.70,128995.45,60923,823579.28,66221.26,17492.56
Appanoose,8427750,83137.76,124862.60,67033,835381.93,61519.26,16252.01
Audubon,2066300,17855.24,26836.31,15202,173849.23,14856.43,3925.72


In [288]:
#https://stackoverflow.com/questions/44908383/how-can-i-group-by-month-from-a-date-field-using-python-pandas

storesmonth2015 = train2015.groupby([train2015['date'].dt.strftime('%B'),
                                     'storenum'])[['bottlevol_ml','statebottlecost',
                                                   'statebottleretail','bottlessold',
                                                   'sale_dollars','volsold_liters',
                                                   'volsold_gallons']].sum().reset_index()
storesmonth2015.head()

,date,storenum,bottlevol_ml,statebottlecost,statebottleretail,bottlessold,sale_dollars,volsold_liters,volsold_gallons
0,April,2106,472675,5510.52,8269.40,10328,145102.74,9736.75,2572.57
1,April,2113,95375,956.81,1438.11,522,6583.76,517.10,136.61
2,April,2130,371125,4219.29,6332.68,7820,116671.10,6919.60,1828.12
3,April,2152,136750,1010.64,1518.38,549,6315.64,579.62,153.10
4,April,2178,256675,2421.43,3635.75,1808,25255.84,1936.12,511.49


In [289]:
monthly2015 = storesmonth2015.pivot(index='storenum',columns='date')
monthly2015.columns

MultiIndex(levels=[['bottlevol_ml', 'statebottlecost', 'statebottleretail', 'bottlessold', 'sale_dollars', 'volsold_liters', 'volsold_gallons'], ['April', 'August', 'December', 'February', 'January', 'July', 'June', 'March', 'May', 'November', 'October', 'September']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]],
           names=[None, 'date'])

In [290]:
monthly2015.head()

bottlevol_ml                                                    \
date            April    August  December  February   January      July   
storenum                                                                  
2106         472675.0  348300.0  439850.0  357800.0  354975.0  388450.0   
2113          95375.0   84750.0  157500.0   86750.0  116750.0   89875.0   
2130         371125.0  257975.0  358350.0  278025.0  279925.0  358550.0   
2152         136750.0  139875.0  125000.0  122625.0  123500.0  158750.0   
2178         256675.0  239100.0  308775.0  195400.0  183425.0  212775.0   

                                                    ...    volsold_gallons  \
date          June     March       May  November    ...           December   
storenum                                            ...                      
2106      369300.0  373150.0  385550.0  325450.0    ...            2337.14   
2113      121125.0  116125.0  119750.0   99500.0    ...             202.90   
2130      279950.0  311625.0  260400.0  277925.0    ...            1660.77   
2152      121000.0  109375.0   97250.0  110000.0    ...             124.51   
2178      282475.0  177375.0  257800.0  215550.0    ...             647.94   

                                                                         \
date     February  January     July     June    March      May November   
storenum                                                                  
2106      1937.11  1880.39  2007.48  1912.15  1946.54  2064.02  1727.98   
2113       125.72   149.68   130.79   172.20   148.33   165.53    98.93   
2130      1554.18  1146.09  1666.08  1330.05  1573.33  1197.74  1405.29   
2152       129.83   128.07   189.75   143.02   115.75   120.10   105.08   
2178       360.06   384.29   454.47   541.36   411.77   593.61   368.75   

                             
date      October September  
storenum                     
2106      2681.97   1943.16  
2113       131.64    152.72  
2130      1631.08   1111.75  
2152       139.99    134.28  
2178       543.71    446.58  

[5 rows x 84 columns]

In [291]:
monthlystores2015 = pd.DataFrame(monthly2015.to_records())
monthlystores2015.head()

,storenum,"('bottlevol_ml', 'April')","('bottlevol_ml', 'August')","('bottlevol_ml', 'December')","('bottlevol_ml', 'February')","('bottlevol_ml', 'January')","('bottlevol_ml', 'July')","('bottlevol_ml', 'June')","('bottlevol_ml', 'March')","('bottlevol_ml', 'May')",...,"('volsold_gallons', 'December')","('volsold_gallons', 'February')","('volsold_gallons', 'January')","('volsold_gallons', 'July')","('volsold_gallons', 'June')","('volsold_gallons', 'March')","('volsold_gallons', 'May')","('volsold_gallons', 'November')","('volsold_gallons', 'October')","('volsold_gallons', 'September')"
0,2106,472675.0,348300.0,439850.0,357800.0,354975.0,388450.0,369300.0,373150.0,385550.0,...,2337.14,1937.11,1880.39,2007.48,1912.15,1946.54,2064.02,1727.98,2681.97,1943.16
1,2113,95375.0,84750.0,157500.0,86750.0,116750.0,89875.0,121125.0,116125.0,119750.0,...,202.90,125.72,149.68,130.79,172.20,148.33,165.53,98.93,131.64,152.72
2,2130,371125.0,257975.0,358350.0,278025.0,279925.0,358550.0,279950.0,311625.0,260400.0,...,1660.77,1554.18,1146.09,1666.08,1330.05,1573.33,1197.74,1405.29,1631.08,1111.75
3,2152,136750.0,139875.0,125000.0,122625.0,123500.0,158750.0,121000.0,109375.0,97250.0,...,124.51,129.83,128.07,189.75,143.02,115.75,120.10,105.08,139.99,134.28
4,2178,256675.0,239100.0,308775.0,195400.0,183425.0,212775.0,282475.0,177375.0,257800.0,...,647.94,360.06,384.29,454.47,541.36,411.77,593.61,368.75,543.71,446.58


In [293]:
monthlystores2015.columns

Index(['storenum', '('bottlevol_ml', 'April')', '('bottlevol_ml', 'August')',
       '('bottlevol_ml', 'December')', '('bottlevol_ml', 'February')',
       '('bottlevol_ml', 'January')', '('bottlevol_ml', 'July')',
       '('bottlevol_ml', 'June')', '('bottlevol_ml', 'March')',
       '('bottlevol_ml', 'May')', '('bottlevol_ml', 'November')',
       '('bottlevol_ml', 'October')', '('bottlevol_ml', 'September')',
       '('statebottlecost', 'April')', '('statebottlecost', 'August')',
       '('statebottlecost', 'December')', '('statebottlecost', 'February')',
       '('statebottlecost', 'January')', '('statebottlecost', 'July')',
       '('statebottlecost', 'June')', '('statebottlecost', 'March')',
       '('statebottlecost', 'May')', '('statebottlecost', 'November')',
       '('statebottlecost', 'October')', '('statebottlecost', 'September')',
       '('statebottleretail', 'April')', '('statebottleretail', 'August')',
       '('statebottleretail', 'December')',
       '('statebottleret

In [294]:
ms_sales2015 = monthlystores2015[['storenum',"('sale_dollars', 'August')", "('sale_dollars', 'December')",
       "('sale_dollars', 'February')", "('sale_dollars', 'January')",
       "('sale_dollars', 'July')", "('sale_dollars', 'June')",
       "('sale_dollars', 'March')", "('sale_dollars', 'May')",
       "('sale_dollars', 'November')", "('sale_dollars', 'October')",
       "('sale_dollars', 'September')"]]
ms_sales2015.head()

,storenum,"('sale_dollars', 'August')","('sale_dollars', 'December')","('sale_dollars', 'February')","('sale_dollars', 'January')","('sale_dollars', 'July')","('sale_dollars', 'June')","('sale_dollars', 'March')","('sale_dollars', 'May')","('sale_dollars', 'November')","('sale_dollars', 'October')","('sale_dollars', 'September')"
0,2106,107201.74,139965.77,115041.01,109795.48,117713.21,110685.44,112330.04,116896.02,97562.38,150977.52,111098.50
1,2113,5400.69,9792.06,6418.05,7685.86,5659.08,9001.75,8247.95,7883.05,5074.57,6279.40,7737.20
2,2130,77098.45,103863.38,104825.96,71307.51,103251.73,90863.09,101630.99,69911.23,94059.28,105598.58,69103.69
3,2152,6608.02,5333.12,5820.09,5746.27,8266.83,6269.00,5238.75,5675.27,4428.02,6202.41,6176.94
4,2178,22256.67,33775.19,16602.12,17970.36,21701.67,26125.18,19838.94,26457.94,20539.60,25118.73,22345.72


In [295]:
print(ms_sales2015.shape)
print(y2015.shape)

(1375, 12)
(1375, 1)


In [296]:
ms_sales2015 = pd.merge(ms_sales2015,y2015.reset_index(),on='storenum')
ms_sales2015.shape

(1375, 13)

In [297]:
ms_sales2015.dropna(inplace=True)
ms_sales2015.shape

(1007, 13)

In [298]:
ms_sales2015.head()

,storenum,"('sale_dollars', 'August')","('sale_dollars', 'December')","('sale_dollars', 'February')","('sale_dollars', 'January')","('sale_dollars', 'July')","('sale_dollars', 'June')","('sale_dollars', 'March')","('sale_dollars', 'May')","('sale_dollars', 'November')","('sale_dollars', 'October')","('sale_dollars', 'September')",sale_dollars
0,2106,107201.74,139965.77,115041.01,109795.48,117713.21,110685.44,112330.04,116896.02,97562.38,150977.52,111098.50,1434369.85
1,2113,5400.69,9792.06,6418.05,7685.86,5659.08,9001.75,8247.95,7883.05,5074.57,6279.40,7737.20,85763.42
2,2130,77098.45,103863.38,104825.96,71307.51,103251.73,90863.09,101630.99,69911.23,94059.28,105598.58,69103.69,1108184.99
3,2152,6608.02,5333.12,5820.09,5746.27,8266.83,6269.00,5238.75,5675.27,4428.02,6202.41,6176.94,72080.36
4,2178,22256.67,33775.19,16602.12,17970.36,21701.67,26125.18,19838.94,26457.94,20539.60,25118.73,22345.72,277987.96


In [300]:
ms_sales2015_X = ms_sales2015[['storenum',"('sale_dollars', 'February')", 
                       "('sale_dollars', 'January')","('sale_dollars', 'March')"]].copy()
ms_sales2015_X['jf'] = ms_sales2015_X["('sale_dollars', 'February')"] * ms_sales2015_X["('sale_dollars', 'January')"]
ms_sales2015_X['fm'] = ms_sales2015_X["('sale_dollars', 'February')"] * ms_sales2015_X["('sale_dollars', 'March')"]

y2015 = ms_sales2015[['storenum','sale_dollars']]

In [302]:
ms_sales2015_X.set_index('storenum',inplace=True)
y2015.set_index('storenum',inplace=True)

In [303]:
model = linear_model.LinearRegression()
model.fit(ms_sales2015_X,y2015)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [305]:
predict2015 = model.predict(ms_sales2015_X)
scores2015 = model.score(ms_sales2015_X,y2015)
print(scores2015)

0.988928866156


In [306]:
print(model.coef_)
print(model.intercept_)

[[  4.41332482e+00   3.30140321e+00   5.94669929e+00   1.12124570e-05
   -1.11176610e-05]]
[-2430.56124572]


In [308]:
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn import metrics

mlr = linear_model.LinearRegression()
scores_kf = cross_val_score(mlr,ms_sales2015_X,y2015,cv=10)

print(scores_kf)
print(np.mean(scores_kf))
print(np.std(scores_kf))

[ 0.97296375  0.9146067   0.93309786  0.97948998  0.9463302   0.95062533
  0.91613353  0.86224133  0.99950582  0.93033863]
0.940533313941
0.0370660968267


In [309]:
predictions2015 = cross_val_predict(mlr,ms_sales2015_X,y2015,cv=10)
r2_2015 = metrics.r2_score(y2015,predictions2015)
r2_2015

0.95566265534964701

In [194]:
def r2_adj(y_true,y_preds,y_mean,p,n):
    sumsqerr = np.sum((y_true - y_preds)**2)
    sumtotsq = np.sum((y_true - y_mean)**2)
    
    numerator = (1 / (n-p-1)) * sumsqerr
    denominator = (1 / (n-1)) * sumtotsq
    
    return 1 - (numerator / denominator)

In [310]:
r2_adj(y2015,predictions2015,np.mean(y2015),5,len(y2015))

sale_dollars    0.955441
dtype: float64

# Try 2014 data

In [331]:
train2014 = pd.read_csv(path+'iowa_liquor_2014.csv')
train2014.head()

,Unnamed: 0,Invoice/Item Number,date,storenum,Store Name,Address,city,zipcode,Store Location,countynum,...,itemnum,itemdescription,Pack,bottlevol_ml,statebottlecost,statebottleretail,bottlessold,sale_dollars,volsold_liters,volsold_gallons
0,108,S17627300143,2014-02-26,2614,Hy-Vee #3 Food and Drugstore,1823 E KIMBERLY RD,DAVENPORT,52807,1823 E KIMBERLY RD\nDAVENPORT 52807\n(41.55678...,82.0,...,15776,Bushmills Irish Whiskey,12,750,$13.00,$19.50,4,$78.00,3.0,0.79
1,112,S18283700037,2014-04-07,2603,Hy-Vee Wine and Spirits / Bettendorf,2890 DEVILS GLEN ROAD,BETTENDORF,52722,2890 DEVILS GLEN ROAD\nBETTENDORF 52722\n(41.5...,82.0,...,30318,Gordon's Gin London Dry - Pet,6,1750,$11.00,$16.49,6,$98.94,10.5,2.77
2,122,S19526900004,2014-06-12,4819,Super Stop 2 / Altoona,2635 ADVENTURELAND DR,ALTOONA,50009,2635 ADVENTURELAND DR\nALTOONA 50009\n(41.6585...,77.0,...,35318,Barton Vodka,6,1750,$6.92,$10.38,6,$62.28,10.5,2.77
3,123,S19153100167,2014-05-23,2515,Hy-Vee Food Store #1 / Mason City,2400 4TH ST SW,MASON CITY,50401,"2400 4TH ST SW\nMASON CITY 50401\n(43.148446, ...",17.0,...,5036,Glenlivet 12 Yr Malt Scotch,12,750,$22.07,$33.11,2,$66.22,1.5,0.40
4,126,S18205200042,2014-04-02,4167,"Iowa Street Market, Inc.",1256 IOWA ST,DUBUQUE,52001,"1256 IOWA ST\nDUBUQUE 52001\n(42.504958, -90.6...",31.0,...,24456,Kessler Blend Whiskey,12,750,$5.51,$8.26,12,$99.12,9.0,2.38


In [332]:
train2014["date"] = pd.to_datetime(train2014["date"])
DataInspect(train2014)

Dataframe Shape: (2097796, 25) 

Duplicate Rows: 0 

Dataframe Types 

 Unnamed: 0                      int64
Invoice/Item Number            object
date                   datetime64[ns]
storenum                        int64
Store Name                     object
Address                        object
city                           object
zipcode                        object
Store Location                 object
countynum                     float64
county                         object
category                      float64
categoryname                   object
vendornum                     float64
Vendor Name                    object
itemnum                         int64
itemdescription                object
Pack                            int64
bottlevol_ml                    int64
statebottlecost                object
statebottleretail              object
bottlessold                     int64
sale_dollars                   object
volsold_liters                float64
volsold_gallons 

statebottleretail : 1748
bottlessold : 232
sale_dollars : 12062
volsold_liters : 509
volsold_gallons : 504


In [333]:
data.rename(columns={'Date':'date', 'Store Number':'storenum', 'City':'city', 
                     'Zip Code':'zipcode', 'County Number':'countynum', 
                     'County':'county',
                     'Category':'category', 'Category Name':'categoryname', 
                     'Vendor Number':'vendornum',
                     'Item Number':'itemnum','Item Description':'itemdescription', 
                     'Bottle Volume (ml)':'bottlevol_ml', 
                     'State Bottle Cost':'statebottlecost',
                     'State Bottle Retail':'statebottleretail', 
                     'Bottles Sold':'bottlessold', 
                     'Sale (Dollars)':'sale_dollars',
                     'Volume Sold (Liters)':'volsold_liters', 
                     'Volume Sold (Gallons)':'volsold_gallons'},inplace=True)

In [334]:
#Remove dollar signs and convert to numeric

bad_col = ['statebottlecost','statebottleretail','sale_dollars']

for item in bad_col:
    train2014[item] = train2014[item].str.replace('$','')
    train2014[item] = train2014[item].apply(pd.to_numeric)
    
train2014.head()

,Unnamed: 0,Invoice/Item Number,date,storenum,Store Name,Address,city,zipcode,Store Location,countynum,...,itemnum,itemdescription,Pack,bottlevol_ml,statebottlecost,statebottleretail,bottlessold,sale_dollars,volsold_liters,volsold_gallons
0,108,S17627300143,2014-02-26,2614,Hy-Vee #3 Food and Drugstore,1823 E KIMBERLY RD,DAVENPORT,52807,1823 E KIMBERLY RD\nDAVENPORT 52807\n(41.55678...,82.0,...,15776,Bushmills Irish Whiskey,12,750,13.00,19.50,4,78.00,3.0,0.79
1,112,S18283700037,2014-04-07,2603,Hy-Vee Wine and Spirits / Bettendorf,2890 DEVILS GLEN ROAD,BETTENDORF,52722,2890 DEVILS GLEN ROAD\nBETTENDORF 52722\n(41.5...,82.0,...,30318,Gordon's Gin London Dry - Pet,6,1750,11.00,16.49,6,98.94,10.5,2.77
2,122,S19526900004,2014-06-12,4819,Super Stop 2 / Altoona,2635 ADVENTURELAND DR,ALTOONA,50009,2635 ADVENTURELAND DR\nALTOONA 50009\n(41.6585...,77.0,...,35318,Barton Vodka,6,1750,6.92,10.38,6,62.28,10.5,2.77
3,123,S19153100167,2014-05-23,2515,Hy-Vee Food Store #1 / Mason City,2400 4TH ST SW,MASON CITY,50401,"2400 4TH ST SW\nMASON CITY 50401\n(43.148446, ...",17.0,...,5036,Glenlivet 12 Yr Malt Scotch,12,750,22.07,33.11,2,66.22,1.5,0.40
4,126,S18205200042,2014-04-02,4167,"Iowa Street Market, Inc.",1256 IOWA ST,DUBUQUE,52001,"1256 IOWA ST\nDUBUQUE 52001\n(42.504958, -90.6...",31.0,...,24456,Kessler Blend Whiskey,12,750,5.51,8.26,12,99.12,9.0,2.38


In [335]:
train2014.drop(['Unnamed: 0','Invoice/Item Number','Store Name','Address','Store Location',
           'vendornum','Vendor Name','Pack'],axis=1,inplace=True)
train2014.shape

(2097796, 17)

In [336]:
train2014.head()

,date,storenum,city,zipcode,countynum,county,category,categoryname,itemnum,itemdescription,bottlevol_ml,statebottlecost,statebottleretail,bottlessold,sale_dollars,volsold_liters,volsold_gallons
0,2014-02-26,2614,DAVENPORT,52807,82.0,Scott,1012300.0,IRISH WHISKIES,15776,Bushmills Irish Whiskey,750,13.00,19.50,4,78.00,3.0,0.79
1,2014-04-07,2603,BETTENDORF,52722,82.0,Scott,1041100.0,AMERICAN DRY GINS,30318,Gordon's Gin London Dry - Pet,1750,11.00,16.49,6,98.94,10.5,2.77
2,2014-06-12,4819,ALTOONA,50009,77.0,Polk,1031080.0,VODKA 80 PROOF,35318,Barton Vodka,1750,6.92,10.38,6,62.28,10.5,2.77
3,2014-05-23,2515,MASON CITY,50401,17.0,Cerro Gordo,1012210.0,SINGLE MALT SCOTCH,5036,Glenlivet 12 Yr Malt Scotch,750,22.07,33.11,2,66.22,1.5,0.40
4,2014-04-02,4167,DUBUQUE,52001,31.0,Dubuque,1011100.0,BLENDED WHISKIES,24456,Kessler Blend Whiskey,750,5.51,8.26,12,99.12,9.0,2.38


In [337]:
#Extract unique City/Zipcode/CountyNum/County tuples from list
#Drop if CountyNum and County are null
#https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/

##change this
counties = train2014[train2014[['city','zipcode','countynum',
                      'county']].duplicated()==False][['city','zipcode',
                                                       'countynum','county']]
counties = counties[counties['countynum'].notnull() & counties['county'].notnull()]
counties['county'].nunique()

99

In [338]:
#Check for duplicated City/Zipcode tuples assigned to multiple counties.
counties[counties.duplicated(['city','zipcode'],False)].sort_values('city')

,city,zipcode,countynum,county
4221,ACKLEY,50601,94.0,Webster
4414,ACKLEY,50601,42.0,Hardin
1,BETTENDORF,52722,82.0,Scott
199,BETTENDORF,52722,48.0,Iowa
2632,CAMBRIDGE,50046,77.0,Polk
41249,CAMBRIDGE,50046,85.0,Story
63,FORT DODGE,50501,94.0,Webster
235,FORT DODGE,50501,42.0,Hardin
40,NORTH LIBERTY,52317,52.0,Johnson
449,NORTH LIBERTY,52317,48.0,Iowa


In [339]:
#Identify list of observations missing Category & CategoryName
#Find matching city/zip pair from COUNTIES list
#If unique observation: replace
#If multiple occurences: hotdeck, randomly choosing value to substitute

random.seed(19871006)
###change this
missing = train2014[np.logical_and(train2014['countynum'].isnull(), train2014['county'].isnull())]

start = len(missing)
print('Missing values:',start)

for i,row in missing.iterrows():
    index = i
    city_missing = row[2]
    zipcode_missing = row[3]
    
    replacement = counties[(counties['city']==city_missing) & 
                           (counties['zipcode']==zipcode_missing)]
    
    if len(replacement) == 1:
        new_countynum = replacement.iat[0,2]
        new_county = replacement.iat[0,3]
    
    elif len(replacement) > 1:
        srs = replacement.sample(1,axis=0)
        new_countynum = srs.iat[0,2]
        new_county = srs.iat[0,3]

    ##verify replacement columns
    ##change dataframe
    if len(replacement) >= 1:
        train2014.iat[index,4] = new_countynum
        train2014.iat[index,5] = new_county

#change this
end = len(train2014[np.logical_and(train2014['countynum'].isnull(), train2014['county'].isnull())])
print('Remaining null values:',end)

Missing values: 1054
Remaining null values: 0


In [340]:
#Annual 2014 totals by store
stores2014 = train2014.groupby('storenum')[['bottlevol_ml','statebottlecost',
                                            'statebottleretail','bottlessold',
                                            'sale_dollars','volsold_liters',
                                            'volsold_gallons']].sum()
stores2014.head()

,bottlevol_ml,statebottlecost,statebottleretail,bottlessold,sale_dollars,volsold_liters,volsold_gallons
storenum,,,,,,,
2106,4509775,49486.72,74317.85,98819,1342161.28,91879.75,24275.65
2113,1274150,13493.85,20274.30,7625,99913.51,7519.85,1986.61
2130,3413550,36542.80,54858.38,68704,1034376.03,61072.50,16134.93
2152,1678450,13688.42,20588.27,7575,94892.90,7917.17,2091.45
2178,2752800,26431.56,39773.99,20621,275606.81,21800.16,5759.34


In [341]:
#2014 County Report
county2014 = train2014.groupby('county')[['bottlevol_ml','statebottlecost',
                                          'statebottleretail','bottlessold',
                                          'sale_dollars','volsold_liters',
                                          'volsold_gallons']].sum()
county2014.head()

,bottlevol_ml,statebottlecost,statebottleretail,bottlessold,sale_dollars,volsold_liters,volsold_gallons
county,,,,,,,
Adair,4409625,38941.52,58551.48,34466,413071.07,33383.60,8817.71
Adams,1599100,16541.87,24845.26,7556,90524.22,6703.57,1769.20
Allamakee,8511650,76980.43,115781.37,59031,795778.07,64852.38,17131.98
Appanoose,7923600,77355.19,116285.26,60099,769647.57,58656.16,15497.30
Audubon,2123350,18183.63,27378.25,14212,168113.60,14952.17,3951.31


In [342]:
#https://stackoverflow.com/questions/44908383/how-can-i-group-by-month-from-a-date-field-using-python-pandas

storesmonth2014 = train2014.groupby([train2014['date'].dt.strftime('%B'),
                                     'storenum'])[['bottlevol_ml','statebottlecost',
                                                   'statebottleretail','bottlessold',
                                                   'sale_dollars','volsold_liters',
                                                   'volsold_gallons']].sum().reset_index()
storesmonth2014.head()

,date,storenum,bottlevol_ml,statebottlecost,statebottleretail,bottlessold,sale_dollars,volsold_liters,volsold_gallons
0,April,2106,362200,3920.03,5885.55,7966,108092.52,7428.45,1962.74
1,April,2113,136500,1434.09,2153.11,792,10321.30,763.98,201.87
2,April,2130,248875,2720.16,4083.91,5650,96518.70,5202.75,1374.52
3,April,2152,134125,1086.05,1632.89,635,8131.35,685.84,181.18
4,April,2178,215650,2094.37,3148.79,1542,20535.05,1638.34,432.80


In [343]:
#http://pandas.pydata.org/pandas-docs/version/0.15/reshaping.html

monthly2014 = storesmonth2014.pivot(index='storenum',columns='date')
monthly2014.columns

MultiIndex(levels=[['bottlevol_ml', 'statebottlecost', 'statebottleretail', 'bottlessold', 'sale_dollars', 'volsold_liters', 'volsold_gallons'], ['April', 'August', 'December', 'February', 'January', 'July', 'June', 'March', 'May', 'November', 'October', 'September']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]],
           names=[None, 'date'])

In [344]:
#https://stackoverflow.com/questions/22779516/unstack-multiindex-dataframe-to-flat-data-frame-in-pandas
monthlystores2014 = pd.DataFrame(monthly2014.to_records())
monthlystores2014.head()

,storenum,"('bottlevol_ml', 'April')","('bottlevol_ml', 'August')","('bottlevol_ml', 'December')","('bottlevol_ml', 'February')","('bottlevol_ml', 'January')","('bottlevol_ml', 'July')","('bottlevol_ml', 'June')","('bottlevol_ml', 'March')","('bottlevol_ml', 'May')",...,"('volsold_gallons', 'December')","('volsold_gallons', 'February')","('volsold_gallons', 'January')","('volsold_gallons', 'July')","('volsold_gallons', 'June')","('volsold_gallons', 'March')","('volsold_gallons', 'May')","('volsold_gallons', 'November')","('volsold_gallons', 'October')","('volsold_gallons', 'September')"
0,2106,362200.0,327575.0,426100.0,351100.0,363175.0,417175.0,311900.0,357250.0,463375.0,...,2285.21,1880.55,1820.15,2231.35,1693.21,1735.58,2499.39,1868.96,2241.60,2147.71
1,2113,136500.0,106650.0,156500.0,71875.0,82250.0,106750.0,133875.0,98625.0,77500.0,...,212.34,127.07,135.38,171.86,238.60,171.64,134.40,137.47,191.71,124.62
2,2130,248875.0,281675.0,377950.0,253825.0,275175.0,329350.0,210725.0,277025.0,316900.0,...,1826.27,1260.07,1098.33,1605.82,1079.49,1331.90,1300.08,1368.93,1444.24,1177.19
3,2152,134125.0,146200.0,147500.0,140500.0,133125.0,141500.0,180875.0,177375.0,138500.0,...,172.89,165.29,166.87,179.91,226.27,217.27,160.42,112.40,144.34,149.34
4,2178,215650.0,228975.0,305200.0,217700.0,166400.0,250050.0,203650.0,233525.0,228775.0,...,672.15,444.42,413.13,553.24,406.91,483.48,491.09,412.68,599.70,414.24


In [345]:
ms_sales2014 = monthlystores2014[['storenum',"('sale_dollars', 'August')", "('sale_dollars', 'December')",
       "('sale_dollars', 'February')", "('sale_dollars', 'January')",
       "('sale_dollars', 'July')", "('sale_dollars', 'June')",
       "('sale_dollars', 'March')", "('sale_dollars', 'May')",
       "('sale_dollars', 'November')", "('sale_dollars', 'October')",
       "('sale_dollars', 'September')"]]
ms_sales2014.head()

,storenum,"('sale_dollars', 'August')","('sale_dollars', 'December')","('sale_dollars', 'February')","('sale_dollars', 'January')","('sale_dollars', 'July')","('sale_dollars', 'June')","('sale_dollars', 'March')","('sale_dollars', 'May')","('sale_dollars', 'November')","('sale_dollars', 'October')","('sale_dollars', 'September')"
0,2106,103890.49,137803.37,103021.34,99578.05,124716.72,90289.61,98742.86,132148.45,105417.33,125045.69,113414.85
1,2113,6720.06,11624.54,6239.05,6502.62,8809.08,11887.98,9249.87,6034.22,6751.13,9396.78,6376.88
2,2130,78059.53,123906.67,82229.22,63054.10,111165.27,63743.25,87695.76,81434.03,86064.86,86441.34,74063.30
3,2152,10187.85,7883.38,7559.78,7125.98,8537.28,10677.40,9545.98,7025.37,5078.19,6014.64,7125.70
4,2178,20673.13,35981.25,19603.54,19457.53,25893.14,19657.54,22638.30,21612.67,20618.22,28383.53,20552.91


In [346]:
##define target vector
y2014 = stores2014[['sale_dollars']]
y2014.head()

,sale_dollars
storenum,
2106,1342161.28
2113,99913.51
2130,1034376.03
2152,94892.90
2178,275606.81


In [347]:
print(ms_sales2014.shape)
print(y2014.shape)

(1319, 12)
(1319, 1)


In [348]:
ms_sales2014 = pd.merge(ms_sales2014,y.reset_index(),on='storenum')
ms_sales2014.shape

(996, 14)

In [349]:
ms_sales2014.isnull().sum()

storenum                          0
('sale_dollars', 'August')       31
('sale_dollars', 'December')      3
('sale_dollars', 'February')     66
('sale_dollars', 'January')      63
('sale_dollars', 'July')         26
('sale_dollars', 'June')         37
('sale_dollars', 'March')        64
('sale_dollars', 'May')          60
('sale_dollars', 'November')     13
('sale_dollars', 'October')      16
('sale_dollars', 'September')    23
index                             0
sale_dollars                      0
dtype: int64

In [350]:
ms_sales2014.dropna(inplace=True)
ms_sales2014.shape

(884, 14)

In [351]:
ms_sales2014.head()

,storenum,"('sale_dollars', 'August')","('sale_dollars', 'December')","('sale_dollars', 'February')","('sale_dollars', 'January')","('sale_dollars', 'July')","('sale_dollars', 'June')","('sale_dollars', 'March')","('sale_dollars', 'May')","('sale_dollars', 'November')","('sale_dollars', 'October')","('sale_dollars', 'September')",index,sale_dollars
0,2106,103890.49,137803.37,103021.34,99578.05,124716.72,90289.61,98742.86,132148.45,105417.33,125045.69,113414.85,0,1434369.85
1,2113,6720.06,11624.54,6239.05,6502.62,8809.08,11887.98,9249.87,6034.22,6751.13,9396.78,6376.88,1,85763.42
2,2130,78059.53,123906.67,82229.22,63054.10,111165.27,63743.25,87695.76,81434.03,86064.86,86441.34,74063.30,2,1108184.99
3,2152,10187.85,7883.38,7559.78,7125.98,8537.28,10677.40,9545.98,7025.37,5078.19,6014.64,7125.70,3,72080.36
4,2178,20673.13,35981.25,19603.54,19457.53,25893.14,19657.54,22638.30,21612.67,20618.22,28383.53,20552.91,4,277987.96


In [352]:
ms_sales2014_X = ms_sales2014[['storenum',"('sale_dollars', 'February')", 
                       "('sale_dollars', 'January')","('sale_dollars', 'March')"]].copy()
ms_sales2014_X['jf'] = ms_sales2014_X["('sale_dollars', 'February')"] * ms_sales2014_X["('sale_dollars', 'January')"]
ms_sales2014_X['fm'] = ms_sales2014_X["('sale_dollars', 'February')"] * ms_sales2014_X["('sale_dollars', 'March')"]

y2014 = ms_sales2014[['storenum','sale_dollars']]

In [353]:
ms_sales2014_X.set_index('storenum',inplace=True)
y2014.set_index('storenum',inplace=True)

In [355]:
print(ms_sales2014_X.shape)
print(y2014.shape)

(884, 5)
(884, 1)


In [356]:
predict2014 = model.predict(ms_sales2014_X)

In [357]:
scores2014 = model.score(ms_sales2014_X,y2014)
print(scores2014)

0.955304717449


In [358]:
print(model.coef_)
print(model.intercept_)

[[  4.41332482e+00   3.30140321e+00   5.94669929e+00   1.12124570e-05
   -1.11176610e-05]]
[-2430.56124572]


In [365]:
print(np.sum(predict2014))
print(np.sum(y2014))
print(np.sum(y2014) - np.sum(predict2014))

237463346.551
sale_dollars    2.544918e+08
dtype: float64
sale_dollars    1.702846e+07
dtype: float64


In [364]:
print(np.sum(y2015))
print(np.sum(predict2015))
print(np.sum(y2015) - np.sum(predict2015))

sale_dollars    2.661978e+08
dtype: float64
266197792.02
sale_dollars    1.192093e-07
dtype: float64


sale_dollars    1.192093e-07
dtype: float64

In [ ]:
# A:

kfcv = LinearRegression()

scores = {}
model = {}

for i in range(2,11):
    scores[i] = cross_val_score(kfcv,X,y,cv=i)
    model[i] = kfcv.fit(X,y)

results = {}

for k,v in scores.items():
    results[k] = [np.mean(v), np.std(v)]
    
collection = {}
for k,v in model.items():
    collection[k] = [v.coef_,v.intercept_]